## Interactive Authentication to OSDU

This notebook is intended to serve as introduction on how to send API requests to OSDU using an refresh_token OAuth 2.0 methodology. The notebook targets the non-production data partition and server. It requires a non-production data partition/environment refresh_token obtained from Postman. If you want to target the production data partition, make sure you obtain a refresh token from the production data partition in Postamn and change the server_id and data_partition_id accordingly.

What is OAuth?
OAuth 2.0, which stands for "Open Authorization," is a standard designed to allow a website or application to access resources hosted by other web applications on behalf of a user.

The notebooks covers the following points:

1. Sending GET and POST request by requesting an access token.
2. Sending GET and POST request using OSDU SDK (OsduTokenCredential).

In [ ]:
"""
REMARK:

This notebook is intended for educative purposes.
Its content will be abstracted from the user when uisng the OSDU SDK and during the OSDU http client class service (5. osdu_http_client.ipynb).
"""

##### 1. Sending GET and POST request by requesting an access token

In [2]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode
from urllib.error import HTTPError
from json import loads
import requests
import os, json

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
body = {
    "client_id": os.environ['token_client_id'],
    "client_secret": os.environ['token_client_secret'],
    "refresh_token": os.environ['token_client_refresh_token_non_prod'],
    "grant_type": "refresh_token"
}

In [5]:
"""
The authentication/authorization requires an application to have been regitered within Azure under
EQN tenancy, which will provide the application with a client_id.

Once the previous is done the application will require to have access to the OSDU main principals
for DEV, TEST, and PROD, so that we can make requests to OSDU using its client_id.

The application can be either a PublicClientApplication or ConfidentialClientApplication.
We will use a ConfidentialClientApplication for this example to first request an access token
using a refresh token and then pass the access token as parameter when sending a request to OSDU.
"""

tenant_id = os.environ['tenant_id']
token_endpoint = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'

body = {
    "client_id": os.environ['token_client_id'],
    "client_secret": os.environ['token_client_secret'],
    "refresh_token": os.environ['token_client_refresh_token_non_prod'],
    "grant_type": "refresh_token"
}

headers = {"Content-Type": "application/x-www-form-urlencoded"}

data = urlencode(body).encode('utf-8')
request = Request(url=token_endpoint, data=data, headers=headers)

with urlopen(request) as response:
    response_body = response.read()
    access_token = loads(response_body)['access_token']

In [6]:

"""
GET REQUEST: Listing legal tags.

To make a request to OSDU, we need to append the access token and also the osdu data partition id
{PROD: "equinor-data", TEST: "npequinor-test", DEV: "npequinor-dev"} we aim to target. 
"""

headers={
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    'Accept': 'application/json',
    "data-partition-id": os.environ['osdu_npdev_data_partition_id'],        
}


"""Sending a GET request"""

resp = requests.request(
    "GET",
    url = f"{os.environ['osdu_np_server']}/legal/v1/legaltags?valid=true",
    headers=headers
)


"""Printing first legal in the response"""

resp.json()['legalTags'][0]

{'name': 'npequinor-dev-equinor-private-default',
 'description': 'A default legaltag to be used for Equinor data',
 'properties': {'countryOfOrigin': ['NO'],
  'contractId': 'Unknown',
  'expirationDate': '9999-12-31',
  'originator': 'Equinor',
  'dataType': 'First Party Data',
  'securityClassification': 'Private',
  'personalData': 'No Personal Data',
  'exportClassification': 'No License Required'}}

In [7]:
"""
POST REQUEST: OSDU aggregated records by schema kind.

To make a request to OSDU, we need to append the access token and also the osdu data partition id
{PROD: "equinor-data", TEST: "npequinor-test", DEV: "npequinor-dev"} we aim to target. 
"""

headers={
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "data-partition-id": os.environ['osdu_npdev_data_partition_id'],        
}

schema_kinds = "*:*:*:*"
payload = payload = {"kind": schema_kinds, 'aggregateBy': 'kind', 'query': "*"}


"""Sending a POST request"""

resp = requests.request(
    "POST",
    url = f"{os.environ['osdu_np_server']}/search/v2/query",
    data = json.dumps(payload),
    headers=headers
)


"""Printing first aggregations in the response"""

resp.json()['aggregations'][:5]

[{'key': 'eqnr:smda-api-v2.0:wellbore-alias:1.0.0', 'count': 747693},
 {'key': 'eqnr:iEnergy-diskos:linesegmentgeometries:1.0.0', 'count': 167619},
 {'key': 'osdu:wks:work-product-component--SeismicLineGeometry:1.0.0',
  'count': 167619},
 {'key': 'eqnr:smda-api-v2.0:wellheaders:1.0.0', 'count': 151456},
 {'key': 'osdu:wks:master-data--Wellbore:1.1.0', 'count': 151048}]

##### 2. Sending GET and POST request using OSDU SDK (OsduTokenCredential).

In [8]:
from osdu.identity import OsduTokenCredential
from osdu.client import OsduClient

In [9]:
"""
Instantiating our OSDU client, which will request an access_token each time we send a request to OSDU.
"""

tenant_id = os.environ['tenant_id']
token_endpoint = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'

credential = OsduTokenCredential(
    client_id=os.environ['token_client_id'],
    client_secret=os.environ['token_client_secret'],
    refresh_token=os.environ['token_client_refresh_token_non_prod'],
    token_endpoint=token_endpoint,
)

client = OsduClient(
    os.environ['osdu_np_server'],
    os.environ['osdu_npdev_data_partition_id'],
    credentials=credential
)

In [10]:
"""
GET REQUEST: Listing legal tags.

Then we make a GET request in the same way as we did before.
"""

resp = client.get_returning_json(
    url=f"{os.environ['osdu_np_server']}/legal/v1/legaltags?valid=true"
)

resp['legalTags'][0]

{'name': 'npequinor-dev-equinor-private-default',
 'description': 'A default legaltag to be used for Equinor data',
 'properties': {'countryOfOrigin': ['NO'],
  'contractId': 'Unknown',
  'expirationDate': '9999-12-31',
  'originator': 'Equinor',
  'dataType': 'First Party Data',
  'securityClassification': 'Private',
  'personalData': 'No Personal Data',
  'exportClassification': 'No License Required'}}

In [11]:
"""
POST REQUEST: OSDU aggregated records by schema kind.

Then we make a POST request in the same way as we did before.
"""

schema_kinds = "*:*:*:*"
payload = payload = {"kind": schema_kinds, 'aggregateBy': 'kind', 'query': "*"}

resp = client.post_returning_json(
    url=f"{os.environ['osdu_np_server']}/search/v2/query",
    data=payload
)

In [12]:
resp['aggregations'][:5]

[{'key': 'eqnr:smda-api-v2.0:wellbore-alias:1.0.0', 'count': 747693},
 {'key': 'eqnr:iEnergy-diskos:linesegmentgeometries:1.0.0', 'count': 167619},
 {'key': 'osdu:wks:work-product-component--SeismicLineGeometry:1.0.0',
  'count': 167619},
 {'key': 'eqnr:smda-api-v2.0:wellheaders:1.0.0', 'count': 151456},
 {'key': 'osdu:wks:master-data--Wellbore:1.1.0', 'count': 151048}]